In [ ]:
# Imports
import openpyxl
import os
import win32com.client as win32

In [ ]:
# Define filepaths (duration - 1 min)

excel_name = "Übersicht EStKöSt-Erklärungen und LV.xlsx" # ACHTUNG Excel ohne Makros verwenden! 
excel_path = os.path.join(os.getcwd(), excel_name)

workbook = openpyxl.load_workbook(excel_path)

# Select the source sheet and the results sheet
source_sheet = workbook["KöSt-Erklärungen 2022"]         # Excel sheet aus dem die Daten genommen werden

# Iterate over rows 4 to 26
for row in range(4, 27):  # 4, 27
    # Check if the value in column F is "offen"
    if source_sheet.cell(row=row, column=6).value == "offen":
        # Get the name, gender, and email from columns J, K, and L
        client = source_sheet.cell(row=row, column=2).value
        gender = source_sheet.cell(row=row, column=11).value
        name = source_sheet.cell(row=row, column=10).value
        email = source_sheet.cell(row=row, column=12).value
        
        # Get the tax year from cell B2
        tax_year = source_sheet.cell(row=2, column=2).value
        
        # Create the email subject
        email_subject = f"KSt {tax_year} DE - {client} - Anfrage Daten"
        
        # Create the email body with the appropriate salutation based on gender
        if gender == "m":
            salutation = f"Sehr geehrter {name},"
        else:
            salutation = f"Sehr geehrte {name},"

        body = f"""<p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>{salutation}<o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'><o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>Wir möchten uns gerne in Bezug auf die deutsche Körperschaftsteuererklärung {tax_year} der {client} mit Ihnen abstimmen.<o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'><o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>Um einen möglichst reibungsfreien Ablauf zu gewährleisten, möchten wir Sie bitten uns die Unterlagen für die laufenden bzw. die heuer begonnenen Betriebsstätten in Deutschland zukommen zu lassen.<o:p></o:p></span></p> 
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'><o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>Gerne stehen wir vorab für Rückfragen zur Verfügung.<o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'><o:p></o:p></span></p>
        <p class=MsoNormal><span lang=EN-US style='font-size:10.0pt;font-family:"Verdana",sans-serif'>Vielen Dank und beste Grüße!<o:p></o:p></span></p>

        """
        
        # Create the email in Outlook using win32

        outlook = win32.Dispatch("outlook.Application")
        mail = outlook.CreateItem(0) 
        mail.Subject = email_subject
        mail.HTMLBody = body
        mail.Sender = "alexander.aufreiter@icon.at"
        mail.To = "alexander.aufreiter@icon.at"
        mail.GetInspector

        #mail.Display()     # opens a separate window for each email
        mail.Save()         # saves the email as draft
        #mail.Send()        # sends the email immediately
        print(f"one mail for {name}")
    else: 
        print(f"no email for you, {name}")

# Close the workbook
workbook.close()
